In [0]:
%pip install xarray netcdf4

In [0]:
%sh pip list

In [0]:
%restart_python
import time
import glob
import os
start = time.time()
era5_path_list = [
    p for p in glob.glob('/Volumes/aer-processed/era5/daily_summary/*.nc')]
print(f'took {time.time()-start:.2f}s to read {len(era5_path_list)} files')


In [0]:
import os
import shutil
import time
import glob
import concurrent.futures

def copy_file(file_path):
    start = time.time()
    size = os.path.getsize(file_path)
    shutil.copy(file_path, target_dir)
    end = time.time()
    return file_path, size, (end - start)

max_workers = 40
target_dir = '/tmp/era5'
offset = 365
index = 0
os.makedirs(target_dir, exist_ok=True)

start = time.time()
era5_path_list = [
    p for p in glob.glob('/Volumes/aer-processed/era5/daily_summary/*.nc')]
print(f'took {time.time()-start:.2f}s to read {len(era5_path_list)} files')

while index < len(era5_path_list):
    batch_start = time.time()
    total_bytes_copied = 0
    files_to_copy = era5_path_list[index : index + offset]

    print(f'max workers: {max_workers} -- ', end='')

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(copy_file, fp) for fp in files_to_copy]

        for f in concurrent.futures.as_completed(futures):
            file_path, file_size, _ = f.result()
            total_bytes_copied += file_size
            elapsed = time.time() - batch_start
            if elapsed > 0:
                bandwidth = (total_bytes_copied / (1024 * 1024)) / elapsed
                gbps = (total_bytes_copied * 8 / (1024 * 1024 * 1024)) / elapsed
            else:
                bandwidth = 0
                gpbs = 0
            print(f'Running bandwidth: {bandwidth:.2f} MB/s ({gbps:.2f} Gbps)')

    batch_elapsed = time.time() - batch_start
    print(f'Copy complete, took {batch_elapsed:.2f}s')
    index += offset

In [0]:
dbutils.fs.ls('dbfs:/tmp/')